In [ ]:
#|default_exp consultas

In [ ]:
#|export
import re
import os
from typing import Union, Iterable
from time import time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import requests
import pandas as pd

from fastcore.basics import store_attr, listify
from fastcore.foundation import L
from fastcore.parallel import parallel
from fastcore.xtras import Path, partialler

from validate_docbr import CPF, CNPJ
from receitaws.requisicoes import Requisicao

d:\Applications\Scoop\apps\miniconda3\current\envs\receitaws\lib\site-packages\fastprogress\fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [ ]:
#|export
def salvar_requisicao(results: Iterable, # Lista com o retorno das requisições
                 saida: str, # Nome do Arquivo de Saída
)->None:
    """Salva a lista de requisições `results` no arquivo `saida`"""
    df = pd.DataFrame(results)
    if saida is None:
        saida = Path.cwd() / 'resultados.csv'
    try:
        saida = Path(saida)
    except TypeError as e:
        raise TypeError("Verifique o caminho do arquivo de saída digitado!") from e

    match saida.suffix:
        case '.csv' | '.txt':
            df.to_csv(saida, index=False)
        case '.xlsx':
            df.to_excel(saida, index=False, engine='openpyxl')
        case '.json':
            df.to_json(saida)
        case '.md':
            df.to_markdown(saida, index=False)
        case '.html':
            df.to_html(saida, index=False)
        case _:
            df.to_csv(saida, index=False)
            
    return df

def requisitar_em_lote(entrada: str, # Arquivo texto de entrada: 1 CPF | CNPJ por linha ou objeto python iterável
                       cpf_usuario: str, # CPF do usuário requisitante
                       tipo: str, # Tipo de Requisição CPF | CNPJ
                       origem: str, # Texto com identificação da requisição: e.g. 'Teste'
                       ambiente: str = 'hm', # Ambiente onde realizar a requisição: hm | pd 
                       cache: int = 36, # Tempo de expiração do cache em meses 
                       saida: str = None, # Arquivo de saída da requisição
                       n_workers: int = 2, # Número de requisições a serem efetuadas em paralelo
)->pd.DataFrame:
    """Lê o arquivo `entrada` com um CPF | CPNJ por linha ou o objeto python iterável.
    Faz a requisição no `ambiente` do receita-ws e salva os resultados em `saida`
    """
    try:
        conteudo = Path(entrada).readlines()
    except TypeError:
        conteudo = listify(entrada)
    req = Requisicao(cpf_usuario, tipo, ambiente, origem, cache)
    resultado = req.consultar_em_lote(conteudo, n_workers)
    return salvar_requisicao(resultado, saida)

In [ ]:
folder = Path.cwd().parent / 'dados'

,Unnamed: 0,Ordem,Tipo_Ident,CNPJ_CPF,Nome_Entidade,Valido
0,0,718,CPF,33481695268,RUBEM MORAIS DE LIMA,True
1,1,838,CPF,47819847034,LUIS ROBERTO CUNHA REY,True
2,2,971,CPF,18876126880,SIDNEY SOUZA DA SILVA,True
3,3,974,CPF,58201343204,VILSON ANTUNES MAXIMIANO,True
4,4,1020,CPF,21996857134,LUIZ CARLOS DA SILVA SANTOS,True


In [ ]:
#| eval: false
cpf_list = pd.read_excel(folder / 'Entidades_AFFO_CPF.xlsx')
cpf_list.head()

In [ ]:
#| eval: false
cnpj = pd.read_excel(folder / 'Entidades_AFFO_CNPJ.xlsx')
cnpj.head()

,Ordem,Tipo_Ident,CNPJ_CPF,Nome_Entidade,Valido
0,1,CNPJ,76535764000143,Oi S.a. - em Recuperacao Judicial,True
1,2,CNPJ,2558157000162,TELEFONICA BRASIL S.A.,True
2,3,CNPJ,2421421000111,TIM S A,True
3,4,CNPJ,40432544000147,CLARO S.A.,True
4,5,CNPJ,43663075000165,WINITY II TELECOM LTDA,True


In [ ]:
#| eval: false
Path(folder / 'cpf.csv').write_text('\n'.join([str(c) for c in cpf_list.head(10).CNPJ_CPF]))

118

In [ ]:
#| eval: false
Path(folder / 'cnpj.csv').write_text('\n'.join([str(c) for c in cnpj.head(10).CNPJ_CPF]))

143

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()